In [13]:
# ✅ Step 1: Upload JSON Key File
from google.colab import files
files.upload()

# ✅ Step 2: Import Required Libraries
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build

# ✅ Step 3: Authenticate Google Docs API
SERVICE_ACCOUNT_FILE = "decoded-plane-452001-m9-4e8641561e8a.json"
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
docs_service = build('docs', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)
print("✅ Google Docs API authenticated successfully!")

# ✅ Step 4: Create a New Google Doc
def create_google_doc(title):
    """Create a new Google Doc with the given title."""
    try:
        doc = docs_service.documents().create(body={"title": title}).execute()
        doc_id = doc['documentId']
        print(f"✅ Google Doc Created: https://docs.google.com/document/d/{doc_id}")
        return doc_id
    except Exception as e:
        print(f"❌ Error creating document: {e}")
        return None

# ✅ Step 5: Create Document and Store ID
doc_id = create_google_doc("Product Team Sync - FINAL FIX")

# ✅ Step 6: Share the Google Doc
def share_document(doc_id, user_email):
    """Shares the created Google Doc with a specific email."""
    try:
        permission = {
            "type": "user",
            "role": "writer",
            "emailAddress": user_email
        }
        drive_service.permissions().create(fileId=doc_id, body=permission, sendNotificationEmail=False).execute()
        print(f"✅ Document shared with {user_email}")
    except Exception as e:
        print(f"❌ Error sharing document: {e}")

your_email = "bhanutejacherupally03@gmail.com"
share_document(doc_id, your_email)

# ✅ Step 7: Replace Document Content Instead of Deleting
def replace_document_content(doc_id):
    """Replaces all document content with a single blank space."""
    try:
        requests = [{"replaceAllText": {"containsText": {"text": "{{.*}}", "matchCase": True}, "replaceText": " "}}]
        docs_service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()
        print("✅ Document replaced with blank space successfully!")
    except Exception as e:
        print(f"❌ Error replacing document content: {e}")

replace_document_content(doc_id)

# ✅ Step 8: Get the Last Valid Index
def get_last_index(doc_id):
    """Fetch the last valid index in the document to avoid out-of-bounds errors."""
    try:
        doc = docs_service.documents().get(documentId=doc_id).execute()
        content = doc.get('body', {}).get('content', [])
        return content[-1]['endIndex'] if content else 1  # Start at 1 if empty
    except Exception as e:
        print(f"❌ Error fetching document index: {e}")
        return 1  # Default to index 1

# ✅ Step 9: Define Markdown Notes
md_text = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

# ✅ Step 10: Convert Markdown to Google Docs Format (Final Fix)
def convert_markdown(md_text, doc_id):
    """Convert Markdown into Google Docs format while ensuring proper insertion."""
    content = []
    index = 1  # Start at index 1

    lines = md_text.strip().split("\n")

    for line in lines:
        if line.startswith("# "):  # Heading 1
            content.append({"insertText": {"location": {"index": index}, "text": line[2:] + "\n\n"}})
            content.append({"updateParagraphStyle": {"range": {"startIndex": index, "endIndex": index + len(line[2:]) + 2}, "paragraphStyle": {"namedStyleType": "HEADING_1"}, "fields": "namedStyleType"}})
            index += len(line[2:]) + 2

        elif line.startswith("## "):  # Heading 2
            content.append({"insertText": {"location": {"index": index}, "text": line[3:] + "\n\n"}})
            content.append({"updateParagraphStyle": {"range": {"startIndex": index, "endIndex": index + len(line[3:]) + 2}, "paragraphStyle": {"namedStyleType": "HEADING_2"}, "fields": "namedStyleType"}})
            index += len(line[3:]) + 2

        elif line.startswith("- [ ]"):  # Checkboxes
            content.append({"insertText": {"location": {"index": index}, "text": "☐ " + line[5:] + "\n"}})
            index += len(line[5:]) + 3

        elif line.startswith("- "):  # Bullet Points
            content.append({"insertText": {"location": {"index": index}, "text": "• " + line[2:] + "\n"}})
            index += len(line[2:]) + 3

        else:  # Regular Text
            content.append({"insertText": {"location": {"index": index}, "text": line + "\n"}})
            index += len(line) + 1

    return content

# ✅ Step 11: Update Google Doc Without Errors
def update_google_doc(doc_id, content):
    """Safely insert formatted content into the Google Doc, ensuring valid indices."""
    try:
        docs_service.documents().batchUpdate(documentId=doc_id, body={"requests": content}).execute()
        print(f"✅ Google Doc Updated Successfully! View here: https://docs.google.com/document/d/{doc_id}")
    except Exception as e:
        print(f"❌ Error updating document: {e}")

# ✅ Step 12: Convert Markdown and Update Google Doc
formatted_content = convert_markdown(md_text, doc_id)
update_google_doc(doc_id, formatted_content)


Saving decoded-plane-452001-m9-4e8641561e8a.json to decoded-plane-452001-m9-4e8641561e8a (29).json
✅ Google Docs API authenticated successfully!
✅ Google Doc Created: https://docs.google.com/document/d/1I6C_PAfozhZjEGuA5nnquwMDxDM2nxl2WnOtB5Xqaw4
✅ Document shared with bhanutejacherupally03@gmail.com
✅ Document replaced with blank space successfully!
✅ Google Doc Updated Successfully! View here: https://docs.google.com/document/d/1I6C_PAfozhZjEGuA5nnquwMDxDM2nxl2WnOtB5Xqaw4
